In [70]:
# Required Libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets, interact

# Step 1: Read the CSV
file_path = '../data/processed/saved_scrapes/mynest_myhome_20240924.csv'
df = pd.read_csv(file_path)

### Step 2: Clean the 'Asking Price' column (Remove currency symbols, commas, etc.)
# Remove any non-numeric characters (commas, currency symbols)
df['Asking Price'] = df['Asking Price'].replace({'[^0-9.]': ''}, regex=True)

### Convert 'Asking Price' and 'MyHome_Floor_Area_Value' to Numeric
df['Asking Price'] = pd.to_numeric(df['Asking Price'], errors='coerce')  # Convert to numeric, invalid values become NaN
df['MyHome_Floor_Area_Value'] = pd.to_numeric(df['MyHome_Floor_Area_Value'], errors='coerce')  # Convert to numeric

### Safe Division: Add Square Metres (Handling NaNs and Division by Zero)
def safe_divide(row):
    if pd.isna(row['Asking Price']) or pd.isna(row['MyHome_Floor_Area_Value']) or row['MyHome_Floor_Area_Value'] == 0:
        return None  # Return None if the value is NaN or the denominator is zero
    else:
        return row['Asking Price'] / row['MyHome_Floor_Area_Value']

# Apply the safe divide function to the DataFrame
df['price_per_square_meter'] = df.apply(safe_divide, axis=1)

# Step 3: Display the DataFrame
df.head(3)  # Displays the first few rows


,Address,Asking Price,Beds,Baths,Property Type,Energy Rating,Eircode,Local Property Tax,Agency Name,Agency Contact,...,MyHome_Latitude,MyHome_Longitude,MyHome_Monthly_Price,MyHome_Floor_Area_Unit,MyHome_Publish_Date,MyHome_Sale_Type,MyHome_Category,MyHome_Featured_Level,MyHome_Link,price_per_square_meter
0,"Taramar, Middle Third, Dublin 5, D05X8N9",750000,4 Bed,1 Bath,End of Terrace,E2,D05 X8N9,€765,Hamill Estate Agents & Valuers,Hamill Estate Agents & Valuers,...,53.373857,-6.203371,0.0,m²,NaN,NaN,NaN,NaN,https://www.myhome.ie/residential/brochure/tar...,5281.690141
1,"87 Haddington Road, Dublin 4, D04WP23",990000,5 Bed,3 Bath,Terrace,SI_666,D04 WP23,"€1,035",Turley Property Advisors,Susan Turley,...,53.335385,-6.239842,0.0,m²,NaN,NaN,NaN,NaN,https://www.myhome.ie/residential/brochure/87-...,5657.142857
2,"Apartment 79, The Northumberlands, Love Lane E...",410000,2 Bed,1 Bath,Apartment,C1,D02 X068,€405,Owen Reilly,Owen Reilly Sales,...,53.349805,-6.260310,0.0,m²,NaN,NaN,NaN,NaN,https://www.myhome.ie/residential/brochure/79-...,6406.250000


In [71]:
import pandas as pd

# Subset of columns, ensuring column name is consistent
columns_subset = ['Address', 'Asking Price', 'Beds', 'Baths', 'Property Type',
                  'Energy Rating', 'Eircode', 'Agency Name', 
                  'MyHome_Latitude', 'MyHome_Longitude', 'price_per_square_meter',
                  'Price Changes', 'MyHome_Floor_Area_Value']

# Subsetting the DataFrame
df_subset = df[columns_subset]

# Filtering rows where 'price_per_square_metre' is greater than 0 (ensure consistent spelling)
df_filtered = df_subset[df_subset['price_per_square_meter'] > 0]

# Display the filtered DataFrame
df_filtered
df = df_filtered
df.head()


,Address,Asking Price,Beds,Baths,Property Type,Energy Rating,Eircode,Agency Name,MyHome_Latitude,MyHome_Longitude,price_per_square_meter,Price Changes,MyHome_Floor_Area_Value
0,"Taramar, Middle Third, Dublin 5, D05X8N9",750000,4 Bed,1 Bath,End of Terrace,E2,D05 X8N9,Hamill Estate Agents & Valuers,53.373857,-6.203371,5281.690141,"Sold, €950,000, Fri Sep 13 2024; Sale Agreed, ...",142.0
1,"87 Haddington Road, Dublin 4, D04WP23",990000,5 Bed,3 Bath,Terrace,SI_666,D04 WP23,Turley Property Advisors,53.335385,-6.239842,5657.142857,"Sold, €1,010,000, Fri Sep 13 2024; Sale Agreed...",175.0
2,"Apartment 79, The Northumberlands, Love Lane E...",410000,2 Bed,1 Bath,Apartment,C1,D02 X068,Owen Reilly,53.349805,-6.260310,6406.250000,"Sold, €480,000, Fri Sep 13 2024; Sale Agreed, ...",64.0
3,"7 Parkside Heath, Clongriffin, Dublin 13, Dubl...",535000,3 Bed,3 Bath,Terrace,A3,D13 WN3C,Sherry FitzGerald Sutton,53.407653,-6.163418,4734.513274,"Sold, €569,000, Fri Sep 13 2024; Unlisted, €53...",113.0
6,"31 Tibradden Grove, Dublin 12, D12P2X4",355000,3 Bed,1 Bath,Terrace,D1,D12 P2X4,Byrne and Moore Property Consultants Limited,53.308228,-6.341920,3349.056604,"Sold, €390,000, Fri Sep 13 2024; Sale Agreed, ...",106.0


In [72]:
import pandas as pd
import re

def get_sold_price_and_date(price_changes: str):
    """
    Extracts the sold asking price and the date sold from the Price Changes column.
    
    Args:
    price_changes (str): The value from the Price Changes column.
    
    Returns:
    tuple: A tuple containing the sold price as a numeric value and the sold date as a string, or (None, None) if not found.
    """
    if isinstance(price_changes, str):  # Ensure the input is a string
        # Updated regex pattern to capture the "Sold" price and date
        match = re.search(r"Sold, €([0-9,]+), [A-Za-z]{3} ([A-Za-z]{3} \d{2} \d{4})", price_changes)
        
        if match:
            # Extract the price, remove any commas, and convert to float
            sold_price = float(match.group(1).replace(',', ''))
            sold_date = match.group(2)  # Extract the date without the day of the week
            return sold_price, sold_date
    
    return None, None

# Apply the function to extract and convert the sold price and sold date for each row using .loc
df.loc[:, 'Sold Asking Price'], df.loc[:, 'Sold Date'] = zip(*df['Price Changes'].apply(get_sold_price_and_date))

# Display the DataFrame
df.head()


/var/folders/bh/0yjbpqfx48z2xb12qyt9nn640000gn/T/ipykernel_41949/1549943581.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Sold Asking Price'], df.loc[:, 'Sold Date'] = zip(*df['Price Changes'].apply(get_sold_price_and_date))
/var/folders/bh/0yjbpqfx48z2xb12qyt9nn640000gn/T/ipykernel_41949/1549943581.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Sold Asking Price'], df.loc[:, 'Sold Date'] = zip(*df['Price Changes'].apply(get_sold_price_and_date))


,Address,Asking Price,Beds,Baths,Property Type,Energy Rating,Eircode,Agency Name,MyHome_Latitude,MyHome_Longitude,price_per_square_meter,Price Changes,MyHome_Floor_Area_Value,Sold Asking Price,Sold Date
0,"Taramar, Middle Third, Dublin 5, D05X8N9",750000,4 Bed,1 Bath,End of Terrace,E2,D05 X8N9,Hamill Estate Agents & Valuers,53.373857,-6.203371,5281.690141,"Sold, €950,000, Fri Sep 13 2024; Sale Agreed, ...",142.0,950000.0,Sep 13 2024
1,"87 Haddington Road, Dublin 4, D04WP23",990000,5 Bed,3 Bath,Terrace,SI_666,D04 WP23,Turley Property Advisors,53.335385,-6.239842,5657.142857,"Sold, €1,010,000, Fri Sep 13 2024; Sale Agreed...",175.0,1010000.0,Sep 13 2024
2,"Apartment 79, The Northumberlands, Love Lane E...",410000,2 Bed,1 Bath,Apartment,C1,D02 X068,Owen Reilly,53.349805,-6.260310,6406.250000,"Sold, €480,000, Fri Sep 13 2024; Sale Agreed, ...",64.0,480000.0,Sep 13 2024
3,"7 Parkside Heath, Clongriffin, Dublin 13, Dubl...",535000,3 Bed,3 Bath,Terrace,A3,D13 WN3C,Sherry FitzGerald Sutton,53.407653,-6.163418,4734.513274,"Sold, €569,000, Fri Sep 13 2024; Unlisted, €53...",113.0,569000.0,Sep 13 2024
6,"31 Tibradden Grove, Dublin 12, D12P2X4",355000,3 Bed,1 Bath,Terrace,D1,D12 P2X4,Byrne and Moore Property Consultants Limited,53.308228,-6.341920,3349.056604,"Sold, €390,000, Fri Sep 13 2024; Sale Agreed, ...",106.0,390000.0,Sep 13 2024


In [73]:
# Check distinct values of property type
distinct_values = df['Property Type'].unique()
distinct_values

array(['End of Terrace', 'Terrace', 'Apartment', 'Semi-D', 'Detached',
       'Duplex', 'Bungalow', 'Studio', 'Townhouse', 'Houses'],
      dtype=object)

In [74]:
# List of property types to filter for (houses)
house_types = ['End of Terrace', 'Terrace', 'Semi-D', 'Detached', 
               'Duplex', 'Bungalow', 'Townhouse', 'Houses']

# Filter rows where 'property_type' is in the house_types list
house_rows = df[df['Property Type'].isin(house_types)]
house_rows.head()

,Address,Asking Price,Beds,Baths,Property Type,Energy Rating,Eircode,Agency Name,MyHome_Latitude,MyHome_Longitude,price_per_square_meter,Price Changes,MyHome_Floor_Area_Value,Sold Asking Price,Sold Date
0,"Taramar, Middle Third, Dublin 5, D05X8N9",750000,4 Bed,1 Bath,End of Terrace,E2,D05 X8N9,Hamill Estate Agents & Valuers,53.373857,-6.203371,5281.690141,"Sold, €950,000, Fri Sep 13 2024; Sale Agreed, ...",142.0,950000.0,Sep 13 2024
1,"87 Haddington Road, Dublin 4, D04WP23",990000,5 Bed,3 Bath,Terrace,SI_666,D04 WP23,Turley Property Advisors,53.335385,-6.239842,5657.142857,"Sold, €1,010,000, Fri Sep 13 2024; Sale Agreed...",175.0,1010000.0,Sep 13 2024
3,"7 Parkside Heath, Clongriffin, Dublin 13, Dubl...",535000,3 Bed,3 Bath,Terrace,A3,D13 WN3C,Sherry FitzGerald Sutton,53.407653,-6.163418,4734.513274,"Sold, €569,000, Fri Sep 13 2024; Unlisted, €53...",113.0,569000.0,Sep 13 2024
6,"31 Tibradden Grove, Dublin 12, D12P2X4",355000,3 Bed,1 Bath,Terrace,D1,D12 P2X4,Byrne and Moore Property Consultants Limited,53.308228,-6.341920,3349.056604,"Sold, €390,000, Fri Sep 13 2024; Sale Agreed, ...",106.0,390000.0,Sep 13 2024
8,"80 Moatfield Road, Coolock, Coolock, Dublin 5,...",395000,3 Bed,1 Bath,Terrace,E1,D05 X9C0,Hamill Estate Agents & Valuers,53.386624,-6.192724,4157.894737,"Sold, €435,000, Thu Sep 12 2024; Sale Agreed, ...",95.0,435000.0,Sep 12 2024


In [80]:
import pandas as pd
import numpy as np

# Assuming df = house_rows is already available
# For sold date assume that we just use everything in the dataset for now. 

# Columns for model fitting
columns_subset_training = ['Address', 'Property Type', 'Beds', 'Baths', 'MyHome_Floor_Area_Value', 'Sold Asking Price', 'MyHome_Latitude', 'MyHome_Longitude'] #'Sold Date',

# Subsetting the DataFrame
df_subset = house_rows[columns_subset_training].copy()  # Use .copy() to avoid SettingWithCopyWarning

# Extract numeric part from 'Beds' and 'Baths' and convert to integer
df_subset['Beds'] = df_subset['Beds'].str.extract('(\d+)').astype(float)  # Extract and convert to float
df_subset['Baths'] = df_subset['Baths'].str.extract('(\d+)').astype(float)  # Extract and convert to float

# Check the updated DataFrame
df_subset.head()


,Address,Property Type,Beds,Baths,MyHome_Floor_Area_Value,Sold Asking Price,MyHome_Latitude,MyHome_Longitude
0,"Taramar, Middle Third, Dublin 5, D05X8N9",End of Terrace,4.0,1.0,142.0,950000.0,53.373857,-6.203371
1,"87 Haddington Road, Dublin 4, D04WP23",Terrace,5.0,3.0,175.0,1010000.0,53.335385,-6.239842
3,"7 Parkside Heath, Clongriffin, Dublin 13, Dubl...",Terrace,3.0,3.0,113.0,569000.0,53.407653,-6.163418
6,"31 Tibradden Grove, Dublin 12, D12P2X4",Terrace,3.0,1.0,106.0,390000.0,53.308228,-6.341920
8,"80 Moatfield Road, Coolock, Coolock, Dublin 5,...",Terrace,3.0,1.0,95.0,435000.0,53.386624,-6.192724


In [115]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Haversine formula to calculate the distance between two points
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])  # Convert degrees to radians
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# Min-Max scaling function for flexible scaling
def min_max_scale(feature):
    min_val = feature.min()
    max_val = feature.max()
    if max_val - min_val == 0:
        return feature - min_val  # Avoid division by zero
    return (feature - min_val) / (max_val - min_val)

# Function to combine Haversine distance and KNN (Euclidean) distance into a final score
def calculate_combined_score(df_subset, given_lat, given_lon, given_beds, given_baths, given_size, 
                            geo_weight=0.3, size_weight=1.0, beds_baths_weight=0.5, max_distance_km=3):
    # Ensure the coordinates are numeric
    df_subset['MyHome_Latitude'] = pd.to_numeric(df_subset['MyHome_Latitude'], errors='coerce')
    df_subset['MyHome_Longitude'] = pd.to_numeric(df_subset['MyHome_Longitude'], errors='coerce')
    
    # Drop rows with missing values in the relevant columns
    df_subset = df_subset.dropna(subset=['MyHome_Floor_Area_Value', 'Beds', 'Baths', 
                                         'MyHome_Latitude', 'MyHome_Longitude'])
    
    # Calculate Haversine distance
    df_subset['Geo Distance (km)'] = df_subset.apply(
        lambda row: haversine_distance(given_lat, given_lon, row['MyHome_Latitude'], row['MyHome_Longitude']), axis=1)
    
    # Apply range filter based on max_distance_km
    df_subset = df_subset[df_subset['Geo Distance (km)'] <= max_distance_km].copy()
    
    # Prepare KNN features
    knn_features = df_subset[['MyHome_Floor_Area_Value', 'Beds', 'Baths']].copy()
    
    # Manually scale each feature using Min-Max scaling
    knn_features_scaled = pd.DataFrame()
    knn_features_scaled['MyHome_Floor_Area_Value'] = min_max_scale(knn_features['MyHome_Floor_Area_Value']) * size_weight
    knn_features_scaled['Beds'] = min_max_scale(knn_features['Beds']) * beds_baths_weight
    knn_features_scaled['Baths'] = min_max_scale(knn_features['Baths']) * beds_baths_weight
    
    # Define the target features
    target_features = pd.DataFrame({
        'MyHome_Floor_Area_Value': [given_size],
        'Beds': [given_beds],
        'Baths': [given_baths]
    })
    
    # Scale target features using the same scaling as knn_features
    target_scaled = pd.DataFrame()
    target_scaled['MyHome_Floor_Area_Value'] = min_max_scale(knn_features['MyHome_Floor_Area_Value']).iloc[0] * size_weight
    target_scaled['Beds'] = min_max_scale(knn_features['Beds']).iloc[0] * beds_baths_weight
    target_scaled['Baths'] = min_max_scale(knn_features['Baths']).iloc[0] * beds_baths_weight
    
    # Alternatively, scale target based on min and max of each feature
    target_scaled = pd.DataFrame({
        'MyHome_Floor_Area_Value': [(given_size - knn_features['MyHome_Floor_Area_Value'].min()) / 
                                     (knn_features['MyHome_Floor_Area_Value'].max() - knn_features['MyHome_Floor_Area_Value'].min()) * size_weight],
        'Beds': [(given_beds - knn_features['Beds'].min()) / 
                 (knn_features['Beds'].max() - knn_features['Beds'].min()) * beds_baths_weight],
        'Baths': [(given_baths - knn_features['Baths'].min()) / 
                  (knn_features['Baths'].max() - knn_features['Baths'].min()) * beds_baths_weight]
    })
    
    # Initialize NearestNeighbors with Euclidean distance
    nbrs = NearestNeighbors(n_neighbors=len(knn_features_scaled), algorithm='auto', metric='euclidean')
    nbrs.fit(knn_features_scaled)
    
    # Find distances from the target to all properties
    knn_distances, indices = nbrs.kneighbors(target_scaled)
    
    # Assign the KNN distances to the dataframe
    df_subset['KNN Distance'] = knn_distances.flatten()
    
    # Combine Haversine distance and KNN (Euclidean) distance using a weighted sum
    df_subset['Combined Score'] = geo_weight * df_subset['Geo Distance (km)'] + (1 - geo_weight) * df_subset['KNN Distance']
    
    # Sort by the combined score
    df_sorted = df_subset.sort_values(by='Combined Score').reset_index(drop=True)
    
    return df_sorted

# Example latitude, longitude, and house features (beds, baths, size)
given_lat = 53.279690  # Example latitude (Blackrock DART station)
given_lon = -6.181159  # Example longitude (Blackrock DART station)
given_beds = 4.0        # Number of beds
given_baths = 2.0       # Number of baths
given_size = 100.0      # Size in square meters
geo_weight = 0.3        # 30% weight for Geo Distance, 70% for KNN Distance
max_distance_km = 3     # Maximum distance of 3 kilometers from the given point
    
# Apply the function to calculate the combined score using Euclidean distance for KNN
df_sorted = calculate_combined_score(
    df_subset=df_subset,
    given_lat=given_lat,
    given_lon=given_lon,
    given_beds=given_beds,
    given_baths=given_baths,
    given_size=given_size,
    geo_weight=geo_weight,
    size_weight=1.0,
    beds_baths_weight=0.5,
    max_distance_km=max_distance_km
)
    
# Display the top rows of the sorted DataFrame
df_sorted.head(20)

/var/folders/bh/0yjbpqfx48z2xb12qyt9nn640000gn/T/ipykernel_41949/3294469650.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['Geo Distance (km)'] = df_subset.apply(


,Address,Property Type,Beds,Baths,MyHome_Floor_Area_Value,Sold Asking Price,MyHome_Latitude,MyHome_Longitude,Geo Distance (km),KNN Distance,Combined Score
0,"Ruane, Newtownpark Avenue, Blackrock, Co. Dubl...",Detached,5.0,2.0,246.0,1600000.0,53.282513,-6.179218,0.339423,0.170797,0.221385
1,"30 Pine Court, Blackrock, Co. Dublin, A94K228",Detached,4.0,3.0,151.0,910000.0,53.283864,-6.177403,0.527000,0.165333,0.273833
2,"9 Avonmore, Foxrock, Dublin 18, D18R9W4",Detached,4.0,3.0,190.0,1215000.0,53.275677,-6.186567,0.573048,0.167433,0.289118
3,"61 Foxrock Avenue, Foxrock, Foxrock, Dublin 18...",Semi-D,4.0,2.0,157.0,875000.0,53.278057,-6.174434,0.482592,0.236245,0.310149
4,"8 Laurleen, Leopardstown, Dublin 18, A94D284",Detached,5.0,3.0,167.8,850000.0,53.279114,-6.191913,0.717864,0.152000,0.321759
5,"18 Foxrock Green, Foxrock, Dublin 18, D18W9T2",Terrace,3.0,3.0,93.0,677500.0,53.275471,-6.172971,0.718631,0.186042,0.345819
6,"9 Pine Court, Newtownpark Avenue, Co Dublin, A...",Semi-D,4.0,3.0,138.0,875000.0,53.286398,-6.176664,0.803476,0.168787,0.359193
7,"28 Belmont Lawn, Stillorgan Road, Blackrock, C...",Detached,4.0,3.0,145.0,980000.0,53.283138,-6.187025,0.546875,0.297059,0.372004
8,"31 Newtown Park, Blackrock, Co. Dublin, A94D6V6",Semi-D,4.0,1.0,110.0,822000.0,53.287447,-6.178787,0.876822,0.205372,0.406807
9,"Iona, 4 Obelisk Grove, Blackrock, Co. Dublin, ...",Semi-D,3.0,3.0,106.0,795000.0,53.288343,-6.184509,0.987644,0.208534,0.442267


In [114]:
# Import Necessary Libraries
import pandas as pd
import matplotlib.pyplot as plt

def analyze_top_n_prices(df, N):
    """
    Analyzes the top N rows of a DataFrame and returns the 25th percentile, median, and 75th percentile
    of the 'Sold Asking Price'. Also plots these statistics.

    Parameters:
    df (pd.DataFrame): The sorted DataFrame.
    N (int): Number of top rows to analyze.

    Returns:
    pd.DataFrame: A DataFrame with the statistics.
    """
    # Validate Input Parameters
    if not isinstance(df, pd.DataFrame):
        raise TypeError("The 'df' parameter must be a pandas DataFrame.")
    if 'Sold Asking Price' not in df.columns:
        raise ValueError("The DataFrame must contain a 'Sold Asking Price' column.")
    if not isinstance(N, int) or N <= 0:
        raise ValueError("The 'N' parameter must be a positive integer.")

    # Select the top N rows
    top_n = df.head(N)

    # Check if there are enough rows
    if top_n.empty:
        print(f"The DataFrame is empty. No data to analyze for top {N} properties.")
        return pd.DataFrame()

    # Calculate the percentiles
    percentiles = top_n['Sold Asking Price'].quantile([0.25, 0.5, 0.75]).rename(index={
        0.25: '25th Percentile',
        0.5: 'Median',
        0.75: '75th Percentile'
        
    })

    # Create a summary DataFrame
    summary_df = pd.DataFrame({
        'Statistic': ['25th Percentile', 'Median', '75th Percentile'],
        'Sold Asking Price (EUR)': [
            percentiles['25th Percentile'], 
            percentiles['Median'], 
            percentiles['75th Percentile']
        ]
    })

    # Print the summary
    print(f"Sold Asking Price Statistics for Top {N} Properties:")
    display(summary_df)

    return summary_df
    # Call the analyze_top_n_prices function
price_summary = analyze_top_n_prices(df_sorted, 20)


Sold Asking Price Statistics for Top 20 Properties:


,Statistic,Sold Asking Price (EUR)
0,25th Percentile,791250.0
1,Median,862500.0
2,75th Percentile,1028750.0
